In [ ]:
import socket
import numpy as np

import sys
import subprocess

def execute(fname):
    """
    # https://www.semicolonworld.com/question/59697/return-value-from-one-python-script-to-another
    s2_out = subprocess.check_output([sys.executable, fname])
    return s2_out
    """
    
    # https://github.com/mynameisvinn/piegrad/blob/master/PieGrad.py
    # https://stackoverflow.com/questions/16877323/getting-return-information-from-another-python-script
    # https://stackoverflow.com/questions/6357361/alternative-to-execfile-in-python-3
    namespace={}  # environment
    with open(fname, "rb") as source_file:
        code = compile(source_file.read(), fname, "exec")
    exec(code, namespace)  # put results in namespace env 
    
    # send numpy results as bytes https://markhneedham.com/blog/2018/04/07/python-serialize-deserialize-numpy-2d-arrays/
    # could picklet
    res = namespace['res']
    res_bytes = res.tobytes()
    return res_bytes
    

def receive_file(c, out_fname):
    """receive file from client and save as out_fname.
    
    https://stackoverflow.com/questions/9382045/send-a-file-through-sockets-in-python
    """
    # data = c.recv(1024).decode('utf-8')

    l = c.recv(1024)
    print("reading", l)
    with open(out_fname,'wb') as f: 
        f.write(l)
    return True

"""
spin up server for remote execution. 

https://www.geeksforgeeks.org/socket-programming-python/
"""

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
endpoint = ("localhost", 3001)
s.bind(endpoint)
s.listen(5)
while True:

    # connect with whoever pinged server
    c, addr = s.accept()      
    print('connected to', addr )

    # receive source file from client
    temp_fname = "barfoo.py"
    _ = receive_file(c, temp_fname)

    # server evaluates source and returns results
    res = execute(temp_fname)  # res is numpy answer in bytes
    print(np.frombuffer(res))

    # return result back to client
    c.send(res)  # res is sent as bytes, so it will have to be converted back to numpy array

    # close connection 
    c.close() 

connected to ('127.0.0.1', 51787)
reading b'# testsdfs\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b)\n\nres = matmul(a, b) * 100'
[ 80.65507938  33.49289304  54.52516847  67.78243939 127.50401394
  70.05093752 112.58028872  46.25018266 141.24549851  76.8079715
 122.37029378 109.90024664 101.43667018  69.19415425 110.79127547
  79.3519587 ]
connected to ('127.0.0.1', 51794)
reading b'# testsdfs\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b)\n\nres = matmul(a, b) * 100'
[116.01588436 104.31876294  75.53531974 123.96933497 126.64311775
  92.04178828 164.62982202 145.92656051 127.36593894 104.48248045
  71.46397768 127.61775089 178.95320866 166.38377677 124.50856074
 198.63390313]
connected to ('127.0.0.1', 51804)
reading b'# testsdfs\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b